## 웹크롤링

### 인터넷 접속 라이브러리 추가
    - 어라라

In [3]:
from urllib.request import urlopen, Request

# 도시별 날씨 탐색 함수
def get_weather(city):
    # 도시별 날씨가 나오는 페이지
    url = 'https://www.weather.go.kr/w/obs-climate/land/city-obs.do'
    page = urlopen(url=url)

    text = page.read().decode('utf-8')
    text = text[text.find(f'>{city}</a>'):]

    # 기온 가져오기 반복문 13번 가능
    for i in range(7):
        text =text[text.find('<td>')+1:]
    
    start = 3
    end = text.find('</td>')
    current_temp = text[start:end]
    print(f'{city}의 현재 기온은 {current_temp}˚C 입니다')

    # 습도 가져오기
    for i in range(3):
        text =text[text.find('<td>')+1:]
    
    start = 3
    end = text.find('</td>')
    current_humid = text[start:end]
    print(f'{city}의 현재 습도 {current_humid}% 입니다')

get_weather('부산')

부산의 현재 기온은 11.6˚C 입니다
부산의 현재 습도 26% 입니다


### openAPI 크롤링

- 공공데이터 포탈
    - https:///www.data.go.kr
    - 부산광역시_버스정보안내기 현황

```python
response = requests.get(total_url,verify=False)
```
- 문제점
    - https를 일반 request로 부르면 SSL 오류 발생
    - 해결하려면 외부모듈 requests를 사용, verify = False 옵션을 지정
    - 아래셀의 19번 라인

In [22]:
import requests
from urllib.parse import quote, unquote, urlencode      # 한글을 URL인코드로 변환하는 함수 -> '%EB%A1%AF%EB%8D%B0%EB%B0%B1%ED%99%94%EC%A0%90'
import json
import ssl

def getDataPortalSearch(stationName, type):
    api_url = 'https://apis.data.go.kr/6260000/BusanTblBusinfoeqStusService/getTblBusinfoeqStusInfo'
    queryString = "?" + urlencode(
        {
            'serviceKey' : '/Ntx3GoQdoc+waMK+X2zeEch5OYmBCAs4wQ74iRnhBGoIIOCyjtbhEszVrYlWbyGT4hBFa6WuPjb8/920+lHHA==',
            'pageNo' : '1',
            'numOfRows' : '10',
            'resultType' : type,
            'stationLoc' : stationName
        }
    )
    total_url = api_url + queryString
    response = requests.get(total_url,verify=False)
    return response.text

try:
    result = getDataPortalSearch('센텀시티','json')
    json_data = json.loads(result)

    station_data = json_data['getTblBusinfoeqStusInfo']['body']['items']['item']

    for item in station_data:
         print(item)
except Exception as e:
    print('찾는 데이터가 없습니다.')


c:\Source\studypython2023\venv\Lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apis.data.go.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


{'stationNum': '09259', 'stationLoc': '센텀시티역.벡스코', 'lat': '35.16974238', 'lng': '129.1327741', 'addr': '해운대구 우2동 1222', 'insYear': '', 'dataDay': '2023-02-03'}
{'stationNum': '09283', 'stationLoc': '롯데백화점센텀시티점', 'lat': '35.16967324', 'lng': '129.1317843', 'addr': '해운대구 우동 1496', 'insYear': '', 'dataDay': '2023-02-03'}
{'stationNum': '09336', 'stationLoc': '롯데백화점센텀시티점', 'lat': '35.16997953', 'lng': '129.1320357', 'addr': '해운대구 우동 1498', 'insYear': '', 'dataDay': '2023-02-03'}
{'stationNum': '09136', 'stationLoc': '센텀시티역.벡스코', 'lat': '35.16951388', 'lng': '129.1332003', 'addr': '해운대구 우2동 1131', 'insYear': '', 'dataDay': '2023-02-03'}


### OpenAPI로 가져온 데이터 지도 표시

```python
stop_str = '<h4>' + item['stationLoc'] + '</h4>' + item['stationNum'] + '<br>' + item['addr']
```

- 문제점
    - 힝..


In [18]:
import folium

if len(station_data) > 0:
    center_lat = station_data[0]['lat']
    center_lag = station_data[0]['lng']
    
m = folium.Map(location=[center_lat,center_lag], zoom_start= 13)

for item in station_data:
    stop_str = '<h4>' + item['stationLoc'] + '</h4>' + item['stationNum'] + '<br>' + item['addr']
    iframe = folium.IFrame(stop_str)
    popup = folium.Popup(iframe, min_width = 200, max_width = 200) # 팝업사이즈 지정
    folium.Marker(location=[item['lat'],item['lng']],popup=popup,
                  icon=folium.Icon(icon='pushpin')).add_to(m)

m

### BeautifulSoup (Version 4)

웹크롤링을 편하게 해주는 도구(라이브러리)

```shell
pip install beautifulsoup4
```

In [23]:
!pip install beautifulsoup4

     -------------------------------------- 129.4/129.4 kB 7.4 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


#### 모듈 import

```python
from bs4 import BeautifulSoup
```

In [26]:
from bs4 import BeautifulSoup
import requests

url = 'https://kin.naver.com/search/list.naver?query=%EC%A7%91%EA%B0%80%EA%B3%A0%EC%8B%B6%EC%96%B4%EC%9A%94'

response = requests.get(url)

if response.status_code == 200: # 웹페이지 정상
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    print(soup)
else:
    print(f'Error : {response.status_code}')


<!DOCTYPE html>

<html lang="ko">
<head>
<meta contents="always" name="referrer"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="집가고싶어요의 지식iN Q&amp;A 검색결과입니다. 궁금증을 해결하지 못했다면 지식iN '질문하기'를 해보세요." name="description">
<meta content="width=1024" name="viewport"/>
<meta content="none" name="msapplication-config">
<link href="https://ssl.pstatic.net/static.kin/static/pc/20230201140250/css/min/common.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/static.kin/static/pc/20230201140250/css/min/components.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/static.kin/static/pc/20230201140250/css/min/other.css" rel="stylesheet" type="text/css"/>
<script>
	
	var pcDomain = "kin.naver.com";
	var pcDomainWithProtocol = "https://kin.naver.com";
	var pcKinServiceProtocol = "https";

	var mobileDomain = "m.kin.naver.com";
	var mobileDomainWithProtocol = "https://m.kin.naver.com";
	var mobileKinServiceProtocol = "https";

	var up

In [32]:
from bs4 import BeautifulSoup
import requests

url = 'https://kin.naver.com/search/list.naver?query=%EC%A7%91%EA%B0%80%EA%B3%A0%EC%8B%B6%EC%96%B4%EC%9A%94'

response = requests.get(url)

if response.status_code == 200: # 웹페이지 정상
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    ul = soup.select_one('ul.basic1')
    titles = ul.select('li > dl > dt > a')
    for title in titles:
        print(title.get_text())
else:
    print(f'Error : {response.status_code}')

집에 있는데 집가고싶어요
집가고싶어요
기숙사에 있는데 집가고싶어요
집 가고 싶어요..... 납치당했어요....
집이 너무 가고싶어요
집 가고싶어요.
쉼터에 가고싶어요
함수 있나요 도와주세요 집에가고싶어요
집에 너무 가고싶어요
이혼소송 중 집에 가고 싶어요.ㅠ.ㅠ
